Import Data


In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
dataset = pd.read_csv("Churn.csv")

In [3]:
# This turns the categorical data into binary/numerical data
x = pd.get_dummies(dataset.drop(["Customer ID", "Churn"], axis=1))
y = dataset["Churn"].apply(lambda val: 1 if val == "Yes" else 0)

In [4]:
# Split into training and testing data
xTrain, xTest, yTrain, yTest = train_test_split(x, y, test_size=0.2)

In [5]:
xTrain.head()

,Senior Citizen,tenure,Monthly Charges,Gender_Female,Gender_Male,Partner_No,Partner_Yes,Dependents_No,Dependents_Yes,Phone Service_No,...,Total Charges_995.35,Total Charges_996.45,Total Charges_996.85,Total Charges_996.95,Total Charges_997.65,Total Charges_997.75,Total Charges_998.1,Total Charges_999.45,Total Charges_999.8,Total Charges_999.9
1755,0,62,74.30,False,True,True,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
4006,0,1,24.05,False,True,True,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
4103,0,7,44.75,False,True,False,True,False,True,False,...,False,False,False,False,False,False,False,False,False,False
1523,0,29,68.85,True,False,True,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
4134,0,56,98.60,False,True,True,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False


In [6]:
yTrain.head()

1755    0
4006    1
4103    0
1523    1
4134    0
Name: Churn, dtype: int64

Import Libraries


In [7]:
import tensorflow
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Activation
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import accuracy_score

Define Model


In [8]:
model = Sequential(
    [
        Dense(units=32, input_dim=len(xTrain.columns)),
        BatchNormalization(),
        Activation("relu"),
        Dropout(0.2),
        Dense(units=64),
        BatchNormalization(),
        Activation("relu"),
        Dropout(0.2),
        Dense(units=1),
        BatchNormalization(),
        Activation("sigmoid"),
    ]
)

In [9]:
# Compile the model
model.compile(loss="binary_crossentropy", optimizer="sgd", metrics=["accuracy"])

In [10]:
# Create callbacks
checkpoint = ModelCheckpoint(
    "model.h5", monitor="val_loss", save_best_only=True, verbose=1
)

callbacks = [EarlyStopping(monitor="val_loss", patience=5, verbose=1), checkpoint]

Train Model


In [11]:
# Train model
xTrain = xTrain.astype("float32")
yTrain = yTrain.astype("float32")

model.fit(
    xTrain,
    yTrain,
    epochs=200,
    batch_size=32,
    callbacks=callbacks,
    verbose=1,
    validation_split=0.1,
)

Epoch 1/200
159/159 [==============================] - ETA: 0s - loss: 0.5848 - accuracy: 0.7129
Epoch 1: val_loss improved from inf to 0.52236, saving model to model.h5
159/159 [==============================] - 1s 4ms/step - loss: 0.5848 - accuracy: 0.7129 - val_loss: 0.5224 - val_accuracy: 0.7695
Epoch 2/200
159/159 [==============================] - ETA: 0s - loss: 0.5319 - accuracy: 0.7456
Epoch 2: val_loss improved from 0.52236 to 0.46457, saving model to model.h5
159/159 [==============================] - 0s 2ms/step - loss: 0.5319 - accuracy: 0.7456 - val_loss: 0.4646 - val_accuracy: 0.7996
Epoch 3/200
151/159 [===========================>..] - ETA: 0s - loss: 0.5042 - accuracy: 0.7608
Epoch 3: val_loss improved from 0.46457 to 0.46106, saving model to model.h5
159/159 [==============================] - 0s 3ms/step - loss: 0.5034 - accuracy: 0.7614 - val_loss: 0.4611 - val_accuracy: 0.7979
Epoch 4/200
158/159 [============================>.] - ETA: 0s - loss: 0.4866 - accuracy:

Test the Model


In [12]:
# Get predicted values
xTest = xTest.astype("float32")

yHat = model.predict(xTest)
yHat = [1 if val > 0.5 else 0 for val in yHat]

45/45 [==============================] - 0s 1ms/step


In [13]:
model = load_model("model.h5")
model.evaluate(xTest, yTest)

45/45 [==============================] - 0s 1ms/step - loss: 0.4781 - accuracy: 0.7637


[0.4781051576137543, 0.7636621594429016]